In [ ]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

import requests
from bs4 import BeautifulSoup
import time
import re
import json
import string
import pandas as pd
import csv
import math
import os
import threading
import logging
import random
from datetime import datetime


!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import re
import json
import string
import pandas as pd
import csv
import math
import os
import threading
import logging
import random
from datetime import datetime

In [ ]:

# targets = ['pcr', 'virus', 'ventilator', 'mysejahtera', 'antigen', 'misinformation', 'covidnow', 'deaths', 'dg', 'noor', 'hisham', 'vaccinated', 'unvaccinated', 'vaccination', 'anti-vaxxers', 'vaccine hesitancy', 'mandate', 'pfizer', 'moderna', 'sinovac', 'az', 'astrazeneca', 'sinopharm', 'mrna', 'ivermectin', 'hydroxychloroquine', 'khairy', 'mco', 'emco', 'cmco', 'rmco', 'covid-19', 'moh', 'distancing', 'mask', 'masking']

The star


In [ ]:
class The_Star_Crawler:
    def __init__(self, targets, sleeptime, directory, verbose):
        if isinstance(targets, list):
            self.targets = targets
        else:
            self.targets = [targets]
        self.sleeptime = sleeptime
        self.verbose = verbose
        self.directory = directory


    def obtainPageNumber(self, target):
        url = f'https://www.thestar.com.my/search?q={target}&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31'
        time.sleep(0.015)
        source = requests.get(url).text
        soup = BeautifulSoup(source, 'lxml')
        regex = r"a*[0-9]+"
        page_navs = soup.find_all(id="slcontent_0_ThisPageRowCounts")
        largest_query = 1
        for page_nav in page_navs:
            page_list = page_nav.text
            page_iter = re.finditer(regex, page_list)
            for match in page_iter:
                largest_query = match.group(0)
        largest_page = math.ceil(float(largest_query)/30)
        print(f'The total number of query result for the target word {target} is {largest_query}, with {largest_page} pages.')

        return largest_page

    def create_project(self, directory):
        if not os.path.exists(directory):
            print(f'Creating project {directory}')
            os.makedirs(directory)

    def crawl(self, largest_page, target, sleeptime):
        csv_file = open(f'{self.directory}/{target}.csv', 'w')
        csv_writer = csv.writer(csv_file)
        if (self.verbose):
            csv_writer.writerow(['headline', 'date', 'time', 'link', 'content'])
        else:
            csv_writer.writerow(['headline', 'date', 'time', 'link'])

        for page in range(1, largest_page):
            print(f'Crawling page {page} of target {target}')
            time.sleep(0.015)
            url = f'https://www.thestar.com.my/search?pgno={page}&q={target}&qguid=&qtag=&QDR=QDR_specific&qsort=oldest&qrec=30&sdate=2019-12-01&edate=2021-12-31'
            source = requests.get(url).text
            soup = BeautifulSoup(source, 'lxml')

            topics = soup.find_all(class_="row list-listing")

            for topic in topics:
                headline = topic.div.h2.a.text
                time_meta = topic.div.span.text
                date_posted = time_meta.split('|')
                date_posted = [item.strip() for item in date_posted]
                reference = topic.div.h2.a.get('href')

                if (self.verbose):
                    content_url = reference
                    time.sleep(0.015)
                    content_source = requests.get(content_url).text
                    content_soup = BeautifulSoup(content_source, 'lxml')

                    content_topics = content_soup.find_all('p')
                    news_body = []
                    ##print(content_topics)

                    for content_topic in content_topics:
                        content_body = content_topic.text
                        #print(content_body)
                        news_body.append(content_body)
                    del news_body[-8:]
                    news_content = "\n".join(news_body).strip()

                    time.sleep(0.4)

                    csv_writer.writerow([headline, date_posted[0], date_posted[1], reference, news_content])
                else:
                    csv_writer.writerow([headline, date_posted[0], date_posted[1], reference])

            time.sleep(sleeptime)

        csv_file.close()


    def run(self, target):
        pageNo = self.obtainPageNumber(target)
        self.crawl(pageNo, target, self.sleeptime)

    def multirun(self):

        self.create_project(self.directory)
        threads = []
        for index, target in enumerate(self.targets):
            logging.info("Main    : create and start thread %d.", index)

            t = threading.Thread(target=self.run, args = (target, ))
            threads.append(t)
            t.start()
        for thr in threads:

            thr.join()



        #!zip -r /content/TheStar.zip /content/TheStar
        #from google.colab import files
        #files.download("/content/TheStar.zip")

In [ ]:
if __name__ == '__main__':
    targets = [['mandate', 'pfizer', 'moderna', 'sinovac', 'astrazeneca','khairy','sinopharm', 'mrna'],
               ['mco','emco', 'cmco', 'rmco', 'covid-19', 'moh', 'distancing'],
               ['mask', 'masking', 'ismail', 'recovery', 'lockdown']]

    for target in targets:
        web_crawler = The_Star_Crawler(target, 1, 'TheStar', True)
        web_crawler.multirun()

In [ ]:
import shutil

shutil.rmtree('/content/TheStar')

Malay mail


In [ ]:
class MalayMailScraper:

    def __init__(self, base_url, relevancy, year, month, slept, sleeptime_mean, sleeptime_var, directory, verbose):
        self.base_url = base_url
        self.relevancy = relevancy
        self.year = year
        self.month = month
        self.slept = slept
        self.sleeptime_mean = sleeptime_mean
        self.sleeptime_var = sleeptime_var
        self.directory = directory
        self.verbose = verbose
        if (self.verbose):
            self.results = pd.DataFrame(data = None, columns = ['Title', 'Year', 'Month', 'Link', 'Content'])
        else:
            self.results = pd.DataFrame(data = None, columns = ['Title', 'Year', 'Month', 'Link'])

    def create_project(self, directory):
        if not os.path.exists(directory):
            print(f'Creating project {directory}')
            os.makedirs(directory)

    def fetch(self, base_url, type, year, month, page, slept, sleeptime_mean, sleeptime_var):
        # Make HTTP GET request
        url = f'{base_url}news/{type}/{year}/{month}?page={page}'
        print(url)
        response = requests.get(url)
        print('type: %s, HTTP GET request to URL: %s | Status code: %s' % (type, response.url, response.status_code))

        # Return HTTP response
        if(slept):
            time.sleep(sleeptime_mean + random.random()*sleeptime_var)
        return response

    def subfetchParse(self, url, slept, sleeptime_mean, sleeptime_var):
        corpus = ''
        if(slept):
            time.sleep(sleeptime_mean/10 + random.random()*sleeptime_var/10)
        response = requests.get(url)
        content = BeautifulSoup(response.text, 'lxml')
        article_data = content.find_all('article')
        if article_data:
            paras = article_data.p.text
            corpus = ' '.join(paras)
        return corpus

    def parse(self, html, year, month, relevancy,  slept, sleeptime_mean, sleeptime_var, verbose):
        '''Parses response's text and extract data from it'''

        # Parse content
        content = BeautifulSoup(html, 'lxml')
        # Extract data
        reference = content.find_all('a', {'class':'py-2 d-block'})
        if reference:
            hyperlink = [link.get('href') for link in reference]
            title = [link.text.lower() for link in reference]

            relevancy_dict = self.relevancy_check(hyperlink, title)
            title_copied = relevancy_dict['titles']
            hyperlink_copied = relevancy_dict['links']

            if (verbose):
                # Loop over the number of entries
                for index in range(0, len(title_copied)):
                    # Append extracted data to results list
                    self.results.append({
                        'Title': title_copied[index],
                        'Year': year,
                        'Month': month,
                        'Link': hyperlink_copied[index]
                    }, ignore_index=True)
            else:
                for index in range(0, len(title_copied)):
                    # Append extracted data to results list
                    corpus = self.subfetchParse(hyperlink_copied[index], slept, sleeptime_mean, sleeptime_var)

                    self.results.append({
                        'Title': title_copied[index],
                        'Year': year,
                        'Month': month,
                        'Link': hyperlink_copied[index],
                        'Content': corpus
                    }, ignore_index=True)

            return True
        else:
            return False

    def relevancy_check(self, relevancy, hyperlink_lst, title_lst):
        hyperlinks = [link for link in hyperlink_lst]
        titles = [title.lower() for title in title_lst]
        delno = 0

        #check for relevancy
        for index in range(len(titles)):
            tokens = title_lst[index].split()
            relevant = False
            for check in relevancy:
                if (check in tokens):
                    relevant = True
                    break
            if not relevant:
                del titles[index-delno]
                del hyperlinks[index-delno]
                delno = delno + 1
            relevant = False

        return {
            'links': hyperlinks,
            'titles': titles,
            'queries': len(titles) - delno
        }

    def write_file(self, directory, excel_filename):
        '''Writes scpared results to CSV file'''

        # Check results list in not empty
        if len(self.results):
            print(f'Writing results to {directory}/{excel_filename}')

            self.results.to_excel(f'/content/{directory}/{excel_filename}')

            print('Done')

    def save_file(self, directory, excel_filename, verbose):
        !zip -r /content/MalayMail.zip /content/MalayMail
        from google.colab import files
        files.download(f"/content/MalayMail.zip")

    def multirun(self):
        types = ['malaysia', 'world']
        years = ['2020', '2021']
        months = ['01', '02' ,'03','04','05','06','07','08','09','10','11','12']

        self.create_project(self.directory)
        # Loop over the range of pages to scrape
        for type in types:
            for year in years:
                for month in months:
                    t = threading.Thread(target=self.run, args = (self.base_url, type, year, month,
                                                                  self.slept, self.sleeptime_mean, self.sleeptime_var,
                                                                  self.verbose,))
                    t.start()
        t.join()
        t.active_threads()

        self.write_file(self.directory, 'MalayMail', self.verbose)
        self.save_file (self.directory, 'MalayMail', self.verbose)

    def run(self, base_url, type, year, month, slept, sleeptime_mean, sleeptime_var, verbose):
        content_exist = True
        page = 1
        while content_exist:
            # Make HTTP GET request
            response = self.fetch(base_url, type, year, month, slept, sleeptime_mean, sleeptime_var)
            # Parse content
            content_exist = self.parse(response.text, year, month, relevancy,
                                       slept, sleeptime_mean, sleeptime_var, verbose)
            page = page + 1


In [ ]:
if __name__ == '__main__':
    random.seed(datetime.now())
    web_crawler = MalayMailScraper()
    web_crawler.run()

In [ ]:
import collections
import dataclasses
import datetime
import email.utils
import enum
import itertools
import json
import random
import logging
import re
import snscrape.base
import string
import time
import typing
import urllib.parse